In [112]:
import pandas as pd
from sqlalchemy import create_engine

In [113]:
bitcoin_file = "Resources/bitcoin.csv"
bitcoin_data_df = pd.read_csv(bitcoin_file)
bitcoin_data_df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
bitcoin_data = bitcoin_data_df.dropna()
bitcoin_data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


In [115]:
bitcoin_data['date'] = pd.to_datetime(bitcoin_data['Timestamp'], unit='s')
bitcoin_data['date_only'] =(bitcoin_data['date']).dt.date

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [116]:
bitcoin_data_2=bitcoin_data
bitcoin_data_2=bitcoin_data_2.drop_duplicates(subset=['date_only'], keep='last')
bitcoin_data_2=bitcoin_data_2.drop(['date','Open','High','Low','Weighted_Price'], axis=1)
bitcoin_data_2=bitcoin_data_2[['Timestamp', 'date_only', 'Close', 'Volume_(BTC)', 'Volume_(Currency)']]
bitcoin_data_2=bitcoin_data_2.rename(columns={'date_only':'Date','Close':'Close(BTC)'})
bitcoin_data_2['Date'] = pd.to_datetime(bitcoin_data_2['Date'])
bitcoin_data_2['BTC_%_Change']= None
bitcoin_data_2.dtypes

Timestamp                     int64
Date                 datetime64[ns]
Close(BTC)                  float64
Volume_(BTC)                float64
Volume_(Currency)           float64
BTC_%_Change                 object
dtype: object

In [117]:
SP_500_data = "Resources/S&P_500.csv"
SP_500_df = pd.read_csv(SP_500_data)
SP_500_df=SP_500_df.round(2)
SP_500_df['Date'] = pd.to_datetime(SP_500_df['Date'])
SP_500_df=SP_500_df.drop(['Open','High','Low','Adj Close'], axis=1)
SP_500_df=SP_500_df.rename(columns={'Volume':'Volume(S&P_500)','Close':'Close(S&P_500)'})

SP_500_df.dtypes


Date               datetime64[ns]
Close(S&P_500)            float64
Volume(S&P_500)             int64
dtype: object

In [118]:
merge_df=pd.merge(bitcoin_data_2,SP_500_df, on = 'Date')

In [119]:
merge_df.head()

,Timestamp,Date,Close(BTC),Volume_(BTC),Volume_(Currency),BTC_%_Change,Close(S&P_500),Volume(S&P_500)
0,1325611620,2012-01-03,5.29,4.010815,21.217210,None,1277.06,3943710000
1,1325699460,2012-01-04,5.57,43.312196,235.747069,None,1277.30,3592580000
2,1325807160,2012-01-05,6.65,0.499266,3.320117,None,1281.06,4315950000
3,1325881320,2012-01-06,6.00,1.000000,6.000000,None,1277.81,3656830000
4,1326149820,2012-01-09,6.30,1.000000,6.300000,None,1280.70,3371600000


In [120]:
merge_df['SP_500_%_Change']= merge_df['Close(S&P_500)'].pct_change()
merge_df['BTC_%_Change']= merge_df['Close(BTC)'].pct_change()

In [121]:
merge_df.head()

,Timestamp,Date,Close(BTC),Volume_(BTC),Volume_(Currency),BTC_%_Change,Close(S&P_500),Volume(S&P_500),SP_500_%_Change
0,1325611620,2012-01-03,5.29,4.010815,21.217210,NaN,1277.06,3943710000,NaN
1,1325699460,2012-01-04,5.57,43.312196,235.747069,0.052930,1277.30,3592580000,0.000188
2,1325807160,2012-01-05,6.65,0.499266,3.320117,0.193896,1281.06,4315950000,0.002944
3,1325881320,2012-01-06,6.00,1.000000,6.000000,-0.097744,1277.81,3656830000,-0.002537
4,1326149820,2012-01-09,6.30,1.000000,6.300000,0.050000,1280.70,3371600000,0.002262


In [122]:
#merge_df['SP_500_%_Change']=(merge_df['SP_500_%_Change'])*100
#merge_df['BTC_%_Change']=merge_df['BTC_%_Change']*100
merge_df['SP_500_%_Change']=merge_df['SP_500_%_Change'].astype(float)
merge_df['BTC_%_Change']=merge_df['BTC_%_Change'].astype(float)

merge_df['SP_500_%_Change']= (merge_df['SP_500_%_Change']*100).map('{0:.2f}%'.format)
merge_df['BTC_%_Change']= (merge_df['BTC_%_Change']*100).map('{0:.2f}%'.format)
merge_df=merge_df.rename(columns={'Timestamp':'Unix_Time', 'Date': 'Market_Date', 'Close(BTC)': 'Close_BTC','Volume_(BTC)': 'Volume_BTC','Volume_(Currency)':'Volume_Currency','BTC_%_Change':'BTC_Per_Change', 'Close(S&P_500)': 'Close_SP_500', 'Volume(S&P_500)': 'Volume_SP_500', 'SP_500_%_Change': 'SP_500_Per_Change'})


In [123]:
merge_df.head()

,Unix_Time,Market_Date,Close_BTC,Volume_BTC,Volume_Currency,BTC_Per_Change,Close_SP_500,Volume_SP_500,SP_500_Per_Change
0,1325611620,2012-01-03,5.29,4.010815,21.217210,nan%,1277.06,3943710000,nan%
1,1325699460,2012-01-04,5.57,43.312196,235.747069,5.29%,1277.30,3592580000,0.02%
2,1325807160,2012-01-05,6.65,0.499266,3.320117,19.39%,1281.06,4315950000,0.29%
3,1325881320,2012-01-06,6.00,1.000000,6.000000,-9.77%,1277.81,3656830000,-0.25%
4,1326149820,2012-01-09,6.30,1.000000,6.300000,5.00%,1280.70,3371600000,0.23%


In [127]:
connection_string = f"root:$2201Persh@localhost/market_db"
engine = create_engine(f'mysql://{connection_string}')

In [128]:
# Confirm tables
engine.table_names()

['market_table']

In [129]:
merge_df.to_sql(name='market_table', con=engine, if_exists='append', index=False)

In [130]:
pd.read_sql_query('select * from market_table', con=engine).head()

,Unix_Time,Market_Date,Close_BTC,Volume_BTC,Volume_Currency,BTC_Per_Change,Close_SP_500,Volume_SP_500,SP_500_Per_Change
0,1325611620,2012-01-03 00:00:00,5.29,4.010810,21.21720,nan%,1277.06,3943710000,nan%
1,1325699460,2012-01-04 00:00:00,5.57,43.312200,235.74700,5.29%,1277.30,3592580000,0.02%
2,1325807160,2012-01-05 00:00:00,6.65,0.499266,3.32012,19.39%,1281.06,4315950000,0.29%
3,1325881320,2012-01-06 00:00:00,6.00,1.000000,6.00000,-9.77%,1277.81,3656830000,-0.25%
4,1326149820,2012-01-09 00:00:00,6.30,1.000000,6.30000,5.00%,1280.70,3371600000,0.23%


In [131]:
pd.read_sql_query('select Unix_Time from market_table', con=engine).head()

,Unix_Time
0,1325611620
1,1325699460
2,1325807160
3,1325881320
4,1326149820
